# Extension of the Lempel-Ziv algorithm to lossy compression

When no *a priori* knowledge on the information source's statistics is available, one can resort to dictionary methods which are another interesting way to perform entropy coding. The main idea resides in exploiting the recurrency of patterns in input data (e.g. the periodicity of a sound wave recorded in an audio track) and collect the most recurring patterns in a **dictionary**, so that every time a subset of the input matches one of the patterns in the dictionary, the index associated with such a pattern is written in the bitstream (rather than all pattern's values). The dictionary is either communicated or known at the decoder side and is updated according to a lockstep fashion.

The following figure depicts the operational mode of a general dictionary method used to encode text associated with palindromes in Italian:

<img src="dictionary-method-schema.png" width="700">

As the input string is processed by the encoder, it is also stored in a dictionary which is a buffer with limited capacity, updated according to a given policy (e.g. FIFO). We note that the substring from the input *ve* has a match with a subset of the characters stored in the dictionary. In this case the encoder will emit a pair <$p$, $l$> indicating the position ($p$) in the buffer where the match starts and its length ($l$). For the example considered the pair would be <1,2>.

Among the many variations of dictionary methods, there is the algorithm developed by Ziv and Lempel in [1977](https://courses.cs.duke.edu/spring03/cps296.5/papers/ziv_lempel_1977_universal_algorithm.pdf) and then improved in [1978](https://www.philippe-fournier-viger.com/spmf/lz78.pdf). The authors show in their paper(s) that the algorithm achieves the same coding efficiency of a code built by knowing the source's statistics (e.g. a Huffman code). Such an asymptotic performance is achieved, provided that the number of input data is sufficiently large. The method performs lossless coding because the match of the input data with any of the patterns stored in the dictionary is meant to be perfect. If such a constraint is relaxed, a lossy version of the Lempel-Ziv method can be obtained whereby the decoder will replace the data read from the input with the closest match present in the dictionary, where the distance/fidelity measure for the match is defined by the designer. We will see in tutorial an application of such extension to the compression of Electro CardioGram (ECG) tracks, collected for the purpose of monitoring the patients' heart rate and other related measurements.

## Preliminaries on ECG signals
An electro cardiogram is a graph showing the eletric (voltage) activity of the heart along time. Due to the heart's cyclic movements of systole and diastole, the ECG track is also periodic with the typical ***PQRST*** complex as depicted in the following image.

<img src="pqrst-example.png" width="400">

The PQRST complex does not repeat exactly throughout the whole measurement but its shape varies depending on several factors, including the heart's working conditions and noise in the measurements to mention a few. The following Python code cell loads into memory an ECG track provided by the [MIT-BIH](https://www.physionet.org/content/mitdb/1.0.0/) dataset and shows a segment to highlight the repetition of the PQRST pattern.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

sampling_rate = 360 # in Hz, that is, number of samples per second
observation_period = 2 # in seconds
sampling_period = sampling_rate * observation_period

# Read the input ECG signal
with open('ecg_data_200_lead_a.bin', 'rb') as fh:
    ecg_track = np.frombuffer(fh.read(), dtype=np.uint8).astype(np.int32)
    
# Plot the ECG track for the observation window selected
time_range = np.linspace(0, observation_period, sampling_period)
plt.figure(figsize=(10,6))
plt.plot(time_range, ecg_track[:sampling_period])
plt.grid()
plt.xlabel('Time [s]', fontsize=16)
plt.ylabel('Amplitude 8 bits (peak = 10 mV)', fontsize=16);

This particular ECG track has been sampled at 360 Hz and the voltage amplitude measured through the leads attached to the patients has been quantised to 8 bits. We note the three PQRST patterns repeating almost every 3/4 of second.

## Design of Lempel-Ziv algorithm for lossy compression of ECG tracks
We've already hinted at the possibility to relax the match of a segment in the ECG track by allowing a tolerance on the similarity the current input has with what is stored in the dictionary. This idea has been originally proposed by [Horpool et al](https://ieeexplore.ieee.org/document/315990). The method provides the use of a circular buffer `B` which will serve the purpose of our dictionary as well as two parameters associated with the minimum and maximum match length `min_match` and `max_match`, respectively. The algorithm would also need some index variables to point at the current position for the input and `B`. At high level, the workflow of the algorithm can be described with the following *Python style* pseudo-code

```python
while input:
    sub_input = read_next_max_match_samples(input)
    length, pos = match_input_with_buffer(sub_input, tolerance)
    if length < min_match:
        output sub_input[0]
    else:
        output pos, length
        copy B[pos:pos + length] to B[current_position:current_position + length]
```

In the pseudo-code above the function `read_next_max_match_samples` would read from the input a chunk of data whose length is the maximum length we allow for a match (i.e. `max_match`). Moreover, `match_input_with_buffer` will move the subset of input data along the buffer `B` and check whether the ***absolute difference*** between each input and buffer sample is less than or equal to the `tolerance` parameter. If this is not the case, the sub set of input data is slided by one position and the (approximate) matching exercises continues.

A vital part of the algorithm is constituted by the copy operation associated with the `else` branch of the pseudo code. At first glance one might think that once a match is found the sub set of input data should be copied to `B` and the encoding can continue. However, those input data loaded into `B` will not be available to the decoder, hence we would have a mismatch in the reconstructed ECG track. Conversely, we should use the buffer data which which were used in the matching but move them at the position currently pointed for `B`.

A final detail worth commenting is that the actual writing of the compressed data into the bitstream should also address the fact the decoder needs to distinguish between *raw* samples, i.e. written in the `if` branch of the workflow and the pairs <$p, l$>. This can be easily done by prefixing the bit string for a *raw* value or *pair* with a single bit.

## Playing with our algorithm
The Python package `ecgcodec` contains all routines needed to implement the encoding of an ECG signal using the lossy Lempel-Ziv algorithm considered in this tutorial. In particular, the method `ecgencoder` will perform the Lempel-Ziv compression when receives as inputs the ECG track, the dictionary, the values for the min and max match and a tolerance threshold which will be used to determine whether input and dictionary samples are close enough to consider them matching. The method doesn't perform the actual bitstream writing but will return a symbolic representation of the compressed data as a Python list of lists whereby each element will contain a list associated with either the raw value or the ECG signal or the position, length pair. Along with the compressed data, the function also returns an array having the same size of the input and containing the samples which the decoder would reconstruct as well as the total number bits used.

In [ ]:
from ecgcodec import ecgencoder

bits_dictionary, tolerance = 12, 4
dictionary = np.zeros((1 << bits_dictionary), np.int32)

compressed_data, reconstructed, total_bits = ecgencoder(ecg_track, dictionary, tolerance)  # Min and max match are default value parameters set to 20 and 255, respectively

The reconstructed ECG track can be plotted against the original to appreciate the differences and the way the algorithm works.

In [ ]:
observation_period = 1 # ins seconds
sampling_period = sampling_rate * observation_period
time_range = np.linspace(0, observation_period, sampling_period)
plt.figure(figsize=(10,6))
plt.plot(time_range, ecg_track[:sampling_period], 'b-', label='Original')
plt.plot(time_range, reconstructed[:sampling_period], 'r-', label=f'Reconstructed - tolerance {tolerance}')
plt.grid()
plt.xlabel('Time [s]', fontsize=16)
plt.ylabel('Amplitude 8 bits (peak = 10 mV)', fontsize=16)
plt.legend();

As may be noted in the graph above, the reconstructed and original tend to diverge more in the areas where the signal flattens: sudden transitions around the PQRST segment have the reconstructed matching more the original, suggesting that the compressor, for the given tolerance value, will tend to match more the bits with small transitions. Worth noting that the degree at which the red and blue curve overlap will change for a larger value of the tolerance.

Now that the reconstructed data and the number of bits spent are available, we want to measure the compression ratio and fidelity of our algorithm so that we should be able to draw a picture later on its rate distortion performance. The compression ratio is relatively easy to compute: it is simply the ratio between the number of bits associated with the input divided by the bits produced by the compressor. For the fidelity measure, the original paper from [Horpool et al](https://ieeexplore.ieee.org/document/315990) proposes a Percentage RMS Difference (PRD) measure where RMS stands for the usual root mean square. More precisely, the PRD is defined as follows:

$$
\large
PRD(X,\hat{X}) = \frac{\sqrt{\sum_i(X_i - \hat{X}_i)^2}}{\sqrt{\sum_i(X_i)^2}} \times 100,
$$

where $X$ denotes the original ECG signal and $\hat{X}$ is its reconstructed counterpart. This fidelity measure is a sort of distance-based whereby the lower the value the closer the reconstructed track is to the original. The following Python code cell computes and shows the values of the compression ratio and PRD.

In [ ]:
cratio = (ecg_track.size * 8) / total_bits
rms_distortion = np.sqrt(np.average(np.square(reconstructed - ecg_track)))
rms_original = np.sqrt(np.average(np.square(ecg_track)))
prd = rms_distortion / rms_original * 100

print(f"Tolerance {tolerance} gives a compression ratio of {cratio:.2f} and PRD of {prd:.2f}%")

The value for PRD and compression ratio depends on the application space. For example, if the signal compressed is then used to detect cardiac arrhythmia, what is important to preserve are the main peaks of the PQRST segment, hence a more aggressive compression may be tolerated in favour of space saved on the device recording the EGR track (e.g. the hard drive used when recording 24 hours worth of the ECG track for the Holter's examination).

We mentioned above that the `ecgencoder` method provides a symbolic representation of the compressed data. To write such a representation onto a binary file, the method `write_compressed_egc` comes to rescue. You can inspect the source code but worth mentioning here that aside from the compressed data, metadata such as the length of the dictionary and the maximum match length needs to be communicated to the decoder so that it knows how much memory to allocate and how many bits read from the bitstream. Note that the input bit depth of an ECG track is hard coded and equal to 8 bits per sample: this can be easily changed and signalled into the bitstream with trivial alterations of the Python code. The following code cell will perform the writing and the decoding to show perfect match between the reconstructed array output by the encoder and the one derived by the decoder.

In [ ]:
from ecgcodec import write_compressed_egc, ecgdecoder
import os

# Convert the compressed data into a bity stream
max_match_bits = 8
write_compressed_egc(compressed_data, 'bitstream.bin', bits_dictionary, max_match_bits)

# Perform the decoding and check perfect alignment with the encoder
with open('bitstream.bin', 'rb') as fh:
    bitstream = np.frombuffer(fh.read(), dtype=np.uint8)

decoded = ecgdecoder(bitstream)

idx_different = np.where(decoded - reconstructed)

# We remove the bitstream.bin file to avoid leaving any byproduct around on the local disk
os.remove('bitstream.bin')

if len(idx_different[0]):
    raise Exception("Encoder and decoder out of sync")
else:
    print("Encoder and decoder are in sync")

Now that we have the whole pipeline in place, we would to assess the rate-distortion performance of the method by varying the value of the tolerance parameter. The following Python cell runs the `ecgencoder` for values of the tolerance in the range \[0 - 7\] inclusive and then compute the PRD and compression ratio. The following Python code cell does this and you need to be patient as it might take a little tad to execute.

In [ ]:
tolerance = np.arange(8)
cratio = np.zeros(tolerance.shape)
prd = np.zeros(tolerance.shape)
original_bits = ecg_track.size * 8

# Generate the results
for idx, t in enumerate(tolerance):
    print(f"Compression with tolerance {t}")
    _, reconstructed, total_bits = ecgencoder(ecg_track, dictionary, t)
    cratio[idx] = original_bits / total_bits
    rms_d = np.sqrt(np.average(np.square(ecg_track - reconstructed)))
    prd[idx] = rms_d / rms_original * 100

# Plot the results
plt.figure(figsize=(10,6))
plt.plot(cratio, prd, 'b-o')
plt.xlabel('Compression ratio', fontsize=16)
plt.ylabel('PRD [%]', fontsize=16)
plt.grid()

The plot above shows how for low values of the tolerance parameter the algorithm is not worth to be employed: tolerance equal to zero would corresponds to lossless coding with a dictionary method which uses more bits than the original input due to the fact that each time we need to add one bit to distinguish between raw data and position-length pairs. In this case one would be better off with conventional methods such as Huffman coding. We remark once again that the noise in the ECG signal makes the matching to fail most of the times which wouldn't be that had the signal been synthetically generated. For tolerance values greater than or equal to three, the algorithm starts to provide the expected coding efficiency.

## Conclusive remarks
In this short tutorial we've explored an interesting approach to extend dictionary methods such the well-known Lempel-Ziv algorithm to perform lossy coding (i.e. compression). The method used is rather simple and its effectiveness can be studied even by varying other parameters such as size of the dictionary, maximum and minimum match length. Finally, worth also mentioning that the approximate matching function can be parallelisation by spawning as many threads as the number of matches identified with the first sample of the current input segment.